In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import DecisionTreeRegressor


In [0]:
IS_SPARK_SUBMIT_CLI = True

if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

#LOAD DATA  -- FareSample.csv and TripSample.csv

In [0]:
file_location = "/FileStore/tables/fareSample.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df1 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
  
file_location = "/FileStore/tables/tripSample.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df2 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:
temp_table_name1 = "fareSample_csv"

df1.createOrReplaceTempView(temp_table_name1)

In [0]:
temp_table_name2 = "tripSample_csv"

df2.createOrReplaceTempView(temp_table_name2)

In [0]:
if IS_SPARK_SUBMIT_CLI:
    fareSample = spark.read.csv('/FileStore/tables/fareSample.csv', inferSchema=True, header=True)
else:
    fareSample = sqlContext.sql("select * from fareSample_csv")

In [0]:
if IS_SPARK_SUBMIT_CLI:
    tripSample = spark.read.csv('/FileStore/tables/tripSample.csv', inferSchema=True, header=True)
else:
    tripSample = sqlContext.sql("select * from tripSample_csv")

#Combine two datasets

In [0]:
fareSample = fareSample.withColumn('id', monotonically_increasing_id())

fareSample.tail(5)


Out[8]: [Row(medallion='503170A5E7444EB68B4C67DD0A56D745', hack_license='873F6A79F298E5FFC2F10C7AECF51699', vendor_id='VTS', pickup_datetime='2013-01-13 14:04:00', payment_type='CSH', fare_amount=7.5, surcharge=0.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, total_amount=8.0, id=17179887662),
 Row(medallion='F6AC5ED8F624EC43788EA84D9979379C', hack_license='BFFE892CFAC8BBB550F2F3B333F3F479', vendor_id='VTS', pickup_datetime='2013-01-13 13:58:00', payment_type='CSH', fare_amount=11.0, surcharge=0.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, total_amount=11.5, id=17179887663),
 Row(medallion='1C7EC6856B82218EDDACD10895B37216', hack_license='07A73D64D585F3B2800778E4F67A9ADB', vendor_id='VTS', pickup_datetime='2013-01-13 14:10:00', payment_type='CRD', fare_amount=4.0, surcharge=0.0, mta_tax=0.5, tip_amount=1.2, tolls_amount=0.0, total_amount=5.7, id=17179887664),
 Row(medallion='474DFB64CE6ED26BB41011B06D35E77D', hack_license='12DF08C467CE0D44897DFB82171CBC63', vendor_id='VTS', pickup_datetime='2013-01-13 14:08:00', payment_type='CRD', fare_amount=5.5, surcharge=0.0, mta_tax=0.5, tip_amount=1.65, tolls_amount=0.0, total_amount=7.65, id=17179887665),
 Row(medallion='A0B0CBCF79CE29AFDE338A116EC7F9A9', hack_license='758AC8FA5B7650FC08F9B327E5880B9E', vendor_id='VTS', pickup_datetime='2013-01-13 13:54:00', payment_type='CRD', fare_amount=14.5, surcharge=0.0, mta_tax=0.5, tip_amount=1.0, tolls_amount=0.0, total_amount=16.0, id=17179887666)]

In [0]:
tripSample = tripSample.withColumn('id', monotonically_increasing_id())

tripSample.tail(5)

Out[9]: [Row(medallion='97F751A427E8A244DB5E8260F8D348E9', hack_license='67A6A619B0F4A87D211F2EC1561B965B', vendor_id='VTS', rate_code=1, store_and_fwd_flag=None, pickup_datetime='2013-01-13 15:04:00', dropoff_datetime='2013-01-13 15:15:00', passenger_count=1, trip_time_in_secs=660, trip_distance=2.06, pickup_longitude=-73.995979, pickup_latitude=40.732788, dropoff_longitude=-73.978401, dropoff_latitude=40.751312, id=17179881325),
 Row(medallion='B79765D48DDE82105BF9018FF5638472', hack_license='837B990B2A90D37AD61B83AFE261E50E', vendor_id='VTS', rate_code=1, store_and_fwd_flag=None, pickup_datetime='2013-01-13 15:09:00', dropoff_datetime='2013-01-13 15:15:00', passenger_count=1, trip_time_in_secs=360, trip_distance=1.51, pickup_longitude=-73.955948, pickup_latitude=40.779053, dropoff_longitude=-73.9701, dropoff_latitude=40.759686, id=17179881326),
 Row(medallion='762411CFDD3FD0DC5B1ACADF7CAFE596', hack_license='57EF38EFD85CF0876DB379C2ADD45134', vendor_id='VTS', rate_code=1, store_and_fwd_flag=None, pickup_datetime='2013-01-13 13:21:00', dropoff_datetime='2013-01-13 13:48:00', passenger_count=1, trip_time_in_secs=1620, trip_distance=2.98, pickup_longitude=-73.978279, pickup_latitude=40.745602, dropoff_longitude=-74.000542, dropoff_latitude=40.7272, id=17179881327),
 Row(medallion='2A850EB9DA47B4058FB0009D662CAC3F', hack_license='57F2D8BEA3EBFD7DB40202B766A27895', vendor_id='VTS', rate_code=1, store_and_fwd_flag=None, pickup_datetime='2013-01-13 13:37:00', dropoff_datetime='2013-01-13 13:46:00', passenger_count=1, trip_time_in_secs=540, trip_distance=1.52, pickup_longitude=-73.987663, pickup_latitude=40.749931, dropoff_longitude=-73.969803, dropoff_latitude=40.756851, id=17179881328),
 Row(medallion='FDD1B352CB83868F656BCE2526608A4D', hack_license='0B186602E0063AD65492CA346E7A3D0F', vendor_id='VTS', rate_code=1, store_and_fwd_flag=None, pickup_datetime='2013-01-13 13:39:00', dropoff_datetime='2013-01-13 13:46:00', passenger_count=1, trip_time_in_secs=420, trip_distance=2.03, pickup_longitude=-73.969513, pickup_latitude=40.763813, dropoff_longitude=-73.984177, dropoff_latitude=40.740101, id=17179881329)]

In [0]:
data=fareSample.join(tripSample,"id")

data.show(5)

+----+--------------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
 id| medallion| hack_license| vendor_id| pickup_datetime| payment_type| fare_amount| surcharge| mta_tax| tip_amount| tolls_amount| total_amount| medallion| hack_license|vendor_id|rate_code|store_and_fwd_flag| pickup_datetime| dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+----+--------------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
 26|0C5296F3C8B16E702...|D2363240A9295EF57...| CMT|2013-01-07 22:39:03| CSH| 7.0| 0.5| 0.5| 0.0| 0.0| 8.0|0C5296F3C8B16E702...|D2363240A9295EF57...| CMT| 1| N|2013-01-07 22:39:03|2013-01-07 22:44:16| 1| 312| 1.5| -73.978653| 40.787735| -73.96579| 40.805321|
 29|312E0CB058D7FC1A6...|7B5156F3899096333...| CMT|2013-01-05 11:54:49| CSH| 7.5| 0.0| 0.5| 0.0| 0.0| 8.0|312E0CB058D7FC1A6...|7B5156F3899096333...| CMT| 1| N|2013-01-05 11:54:49|2013-01-05 12:03:48| 1| 539| 0.8| -73.977127| 40.74831| -73.990913| 40.751053|
 474|889230A3561FAED35...|D286E47EFB47B6877...| VTS|2013-01-13 11:02:00| CSH| 6.5| 0.0| 0.5| 0.0| 0.0| 7.0|889230A3561FAED35...|D286E47EFB47B6877...| VTS| 1| null|2013-01-13 11:02:00|2013-01-13 11:08:00| 1| 360| 1.36| -73.989616| 40.757286| -73.980301| 40.761314|
 964|9A84FA618F78C840B...|28AE81497962D39D8...| VTS|2013-01-13 11:00:00| CRD| 14.0| 0.0| 0.5| 2.8| 0.0| 17.3|9A84FA618F78C840B...|28AE81497962D39D8...| VTS| 1| null|2013-01-13 11:00:00|2013-01-13 11:14:00| 1| 840| 4.08| -74.009789| 40.712929| -73.973923| 40.758816|
1677|B51E373D4B4CB185F...|559946CB283643A1C...| VTS|2013-01-13 10:28:00| CRD| 9.5| 0.0| 0.5| 2.0| 0.0| 12.0|B51E373D4B4CB185F...|559946CB283643A1C...| VTS| 1| null|2013-01-13 10:28:00|2013-01-13 10:37:00| 6| 540| 2.33| -73.997734| 40.756573| -73.991104| 40.735676|
+----+--------------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
only showing top 5 rows

#Select Data

In [0]:
data1 = data.select('passenger_count', 'trip_time_in_secs', 'trip_distance', col(' fare_amount').alias('label'))

In [0]:
dataaf = data1.dropna(how = 'any')

In [0]:
dataaf.show(5)

+---------------+-----------------+-------------+-----+
passenger_count|trip_time_in_secs|trip_distance|label|
+---------------+-----------------+-------------+-----+
 4| 382| 1.0| 6.5|
 1| 259| 1.5| 6.0|
 1| 282| 1.1| 5.5|
 2| 244| 0.7| 5.0|
 1| 560| 2.1| 9.5|
+---------------+-----------------+-------------+-----+
only showing top 5 rows

#Setup Train and Test datasets

In [0]:
splits = dataaf.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

#Boosted Decision Tree Regression

In [0]:
assembler = VectorAssembler(inputCols = ['passenger_count', 'trip_time_in_secs', 'trip_distance'], outputCol="features")
gbt = GBTRegressor(labelCol="label")

In [0]:
paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [2, 3])\
  .addGrid(gbt.maxIter, [10, 20])\
  .build()
  
##evaluator = RegressionEvaluator(metricName="r2", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())

cv = CrossValidator(estimator=gbt, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid)

In [0]:
pipeline = Pipeline(stages=[assembler, cv])
pipelineModel = pipeline.fit(train)

/databricks/spark/python/pyspark/ml/util.py:800: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [0]:
predictions = pipelineModel.transform(test)

In [0]:
predictions.show(5)

+---------------+-----------------+-------------+---------+-----------------+------------------+
passenger_count|trip_time_in_secs|trip_distance|trueLabel| features| prediction|
+---------------+-----------------+-------------+---------+-----------------+------------------+
 0| 1020| 5.21| 50.0|[0.0,1020.0,5.21]|14.866251153885319|
 1| 0| 0.0| 2.5| [1.0,0.0,0.0]| 13.87777690325728|
 1| 0| 0.0| 2.5| [1.0,0.0,0.0]| 13.87777690325728|
 1| 0| 0.0| 2.5| [1.0,0.0,0.0]| 13.87777690325728|
 1| 0| 0.0| 2.5| [1.0,0.0,0.0]| 13.87777690325728|
+---------------+-----------------+-------------+---------+-----------------+------------------+
only showing top 5 rows

In [0]:
predicted = predictions.select("features", "prediction", "trueLabel")
predicted.show(10)

+-----------------+------------------+---------+
 features| prediction|trueLabel|
+-----------------+------------------+---------+
[0.0,1020.0,5.21]|14.866251153885319| 50.0|
 [1.0,0.0,0.0]| 13.87777690325728| 2.5|
 [1.0,0.0,0.0]| 13.87777690325728| 2.5|
 [1.0,0.0,0.0]| 13.87777690325728| 2.5|
 [1.0,0.0,0.0]| 13.87777690325728| 2.5|
 [1.0,0.0,0.0]| 13.87777690325728| 2.5|
 [1.0,0.0,0.0]| 13.87777690325728| 2.5|
 [1.0,0.0,0.0]| 13.87777690325728| 4.5|
 [1.0,0.0,0.0]| 13.87777690325728| 4.5|
 [1.0,0.0,0.0]| 13.87777690325728| 7.0|
+-----------------+------------------+---------+
only showing top 10 rows

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
dataPred = spark.sql("SELECT trueLabel, prediction FROM regressionPredictions")

dataPred.show(5)

+---------+------------------+
trueLabel| prediction|
+---------+------------------+
 50.0|14.866251153885319|
 2.5| 13.87777690325728|
 2.5| 13.87777690325728|
 2.5| 13.87777690325728|
 2.5| 13.87777690325728|
+---------+------------------+
only showing top 5 rows

#RMSE/R2 for GBT-Regression

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print( "Root Mean Square Error (RMSE) for GBT Regression :", rmse)

Root Mean Square Error (RMSE) for GBT Regression : 10.205602402118904

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print( "Coefficient of Determination (R2) for GBT Regression :", r2)

Coefficient of Determination (R2) for GBT Regression : 0.13425932181419653

#Linear Regression

In [0]:
assembler = VectorAssembler(inputCols = ['passenger_count', 'trip_time_in_secs', 'trip_distance'], outputCol="features")
lr = LinearRegression(labelCol="label",featuresCol="features", maxIter=10, regParam=0.3)
pipeline1 = Pipeline(stages=[assembler, lr])

In [0]:
paramGrid1 = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.01]).addGrid(lr.maxIter, [10, 5]).build()
trainval = TrainValidationSplit(estimator=pipeline1, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid1, trainRatio=0.8)

In [0]:
pipelineModel = trainval.fit(train)

In [0]:
predictions = pipelineModel.transform(test)

In [0]:
predicted = predictions.select("features", "prediction", "trueLabel")
predicted.show(10)

+-----------------+------------------+---------+
 features| prediction|trueLabel|
+-----------------+------------------+---------+
[0.0,1020.0,5.21]|14.638972783290274| 50.0|
 [1.0,0.0,0.0]| 8.535028340151294| 2.5|
 [1.0,0.0,0.0]| 8.535028340151294| 2.5|
 [1.0,0.0,0.0]| 8.535028340151294| 2.5|
 [1.0,0.0,0.0]| 8.535028340151294| 2.5|
 [1.0,0.0,0.0]| 8.535028340151294| 2.5|
 [1.0,0.0,0.0]| 8.535028340151294| 2.5|
 [1.0,0.0,0.0]| 8.535028340151294| 4.5|
 [1.0,0.0,0.0]| 8.535028340151294| 4.5|
 [1.0,0.0,0.0]| 8.535028340151294| 7.0|
+-----------------+------------------+---------+
only showing top 10 rows

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
dataPred = spark.sql("SELECT trueLabel, prediction FROM regressionPredictions")

dataPred.show(5)

+---------+------------------+
trueLabel| prediction|
+---------+------------------+
 50.0|14.638972783290274|
 2.5| 8.535028340151294|
 2.5| 8.535028340151294|
 2.5| 8.535028340151294|
 2.5| 8.535028340151294|
+---------+------------------+
only showing top 5 rows

#RMSE/R2 for Linear Regression

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print( "Root Mean Square Error (RMSE) for Linear Regression :", rmse)

Root Mean Square Error (RMSE) for Linear Regression : 10.279668474240642

In [0]:
evaluator1  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="r2")
r2 = evaluator1.evaluate(predictions)
print( "Coefficient of Determination (R2) for Linear Regression :", r2)

Coefficient of Determination (R2) for Linear Regression : 0.12164768198991038

#Decision Forest Regression

In [0]:
assembler = VectorAssembler(inputCols = ['passenger_count', 'trip_time_in_secs', 'trip_distance'], outputCol="features")
dt = DecisionTreeRegressor(labelCol="label",featuresCol="features")

In [0]:
paramGrid2 = ParamGridBuilder()\
  .addGrid(dt.maxDepth, [2,3])\
  .addGrid(dt.maxBins, [10,20])\
  .build()

In [0]:
#evaluator = RegressionEvaluator(metricName="rmse", labelCol=dt.getLabelCol(), predictionCol=dt.getPredictionCol())

dtcv = CrossValidator(estimator = dt, estimatorParamMaps = paramGrid2, evaluator = RegressionEvaluator(), numFolds=2)

In [0]:
pipeline2 = Pipeline(stages=[assembler, dtcv])
pipelineModel = pipeline2.fit(train)

In [0]:
predictions = pipelineModel.transform(test)

In [0]:
predicted = predictions.select("features","prediction","truelabel")
predicted.show(10)

+-----------------+------------------+---------+
 features| prediction|truelabel|
+-----------------+------------------+---------+
[0.0,1020.0,5.21]|15.306746987951808| 50.0|
 [1.0,0.0,0.0]| 9.438852935349665| 2.5|
 [1.0,0.0,0.0]| 9.438852935349665| 2.5|
 [1.0,0.0,0.0]| 9.438852935349665| 2.5|
 [1.0,0.0,0.0]| 9.438852935349665| 2.5|
 [1.0,0.0,0.0]| 9.438852935349665| 2.5|
 [1.0,0.0,0.0]| 9.438852935349665| 2.5|
 [1.0,0.0,0.0]| 9.438852935349665| 4.5|
 [1.0,0.0,0.0]| 9.438852935349665| 4.5|
 [1.0,0.0,0.0]| 9.438852935349665| 7.0|
+-----------------+------------------+---------+
only showing top 10 rows

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
dataPred = spark.sql("SELECT trueLabel, prediction FROM regressionPredictions")

dataPred.show(5)

+---------+------------------+
trueLabel| prediction|
+---------+------------------+
 50.0|15.306746987951808|
 2.5| 9.438852935349665|
 2.5| 9.438852935349665|
 2.5| 9.438852935349665|
 2.5| 9.438852935349665|
+---------+------------------+
only showing top 5 rows

#RMSE/R2 for Decision Forest Regression

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print( "Root Mean Square Error (RMSE) for Decision Forest Regression :", rmse)

Root Mean Square Error (RMSE) for Decision Forest Regression : 10.290313716291786

In [0]:
evaluator1  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="r2")
r2 = evaluator1.evaluate(predictions)
print( "Coefficient of Determination (R2) for Decision Forest Regression :", r2)

Coefficient of Determination (R2) for Decision Forest Regression : 0.11982756211888457